In [1]:
from utils import processed, load, denormalize, get_normalized_adj, generate_dataset
import torch

In [2]:
num_timesteps_input = 6 # 30 minutes
num_timesteps_output = 2 # 10 minutes

In [3]:
raw_trunc_dir = "./data/raw/trunc/"
process_dir = "./data/processed/"

# overwrite = False means that the processing function will only run if the process data files do not exist
# overwrite = True => functions will run regardless
processed(raw_trunc_dir, process_dir, overwrite=False)
A, X, metadata, means, stds = load(process_dir)

split_line1 = int(X.shape[2] * 0.6)
split_line2 = int(X.shape[2] * 0.8)

train_original_data = X[:, :, :split_line1]
val_original_data = X[:, :, split_line1:split_line2]
test_original_data = X[:, :, split_line2:]

training_input, training_target = generate_dataset(train_original_data,
                                                   num_timesteps_input=num_timesteps_input,
                                                   num_timesteps_output=num_timesteps_output)
val_input, val_target = generate_dataset(val_original_data,
                                         num_timesteps_input=num_timesteps_input,
                                         num_timesteps_output=num_timesteps_output)
test_input, test_target = generate_dataset(test_original_data,
                                           num_timesteps_input=num_timesteps_input,
                                           num_timesteps_output=num_timesteps_output)
# input shape (num_samples ,num_vertices, num_timesteps_window, num_features)
# output shape (num_samples ,num_vertices, num_timesteps_window)
A_wave = get_normalized_adj(A)
A_wave = torch.from_numpy(A_wave)

Processing ./data/raw/trunc/Mon_Mar_8_2021\22_10_08.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\22_15_09.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\22_20_10.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\22_25_07.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\22_30_09.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\22_35_20.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\22_40_08.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\22_45_09.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\22_50_08.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\22_55_12.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\23_00_08.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\23_05_09.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\23_10_09.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\23_15_08.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\23_20_10.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\23_25_10.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\23_30_09.json
Processing ./data/raw/trunc/Mon

Processing ./data/raw/trunc/Tue_Mar_9_2021\10_50_09.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\10_55_19.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\11_00_11.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\11_05_09.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\11_10_09.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\11_15_08.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\11_20_09.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\11_25_11.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\11_30_08.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\11_35_09.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\11_40_19.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\11_45_09.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\11_50_08.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\11_55_10.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\12_00_09.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\12_05_08.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\12_10_08.json
Processing ./data/raw/trunc/Tue

Processing ./data/raw/trunc/Wed_Mar_10_2021\02_55_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\03_00_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\03_05_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\03_10_11.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\03_15_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\03_20_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\03_25_19.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\03_30_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\03_35_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\03_40_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\03_45_10.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\03_50_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\03_55_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\04_00_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\04_05_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\04_10_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\04_15_09.json
Processing ./d

Processing ./data/raw/trunc/Wed_Mar_10_2021\16_55_20.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\17_00_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\17_05_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\17_10_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\17_15_08.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\17_20_11.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\17_25_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\17_30_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\17_35_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\17_40_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\17_45_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\17_50_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\17_55_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\18_00_19.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\18_05_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\18_10_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\18_15_12.json
Processing ./d

Processing ./data/raw/trunc/Thu_Mar_11_2021\05_15_10.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\05_20_09.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\05_25_18.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\05_30_09.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\05_35_09.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\05_40_09.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\05_45_08.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\05_50_09.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\05_55_09.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\06_00_08.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\06_05_09.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\06_10_08.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\06_15_08.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\06_20_09.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\06_25_08.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\06_30_09.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\06_35_09.json
Processing ./d

...

Processing ./data/raw/trunc/Thu_Apr_1_2021\12_30_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\12_35_08.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\12_40_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\12_45_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\12_50_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\12_55_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\13_00_10.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\13_05_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\13_10_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\13_15_08.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\13_20_19.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\13_25_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\13_30_08.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\13_35_08.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\13_40_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\13_45_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\13_50_09.json
Processing ./data/raw/trunc/Thu

Processing ./data/raw/trunc/Fri_Apr_2_2021\03_40_08.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\03_45_10.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\03_50_12.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\03_55_08.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\04_00_09.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\04_05_09.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\04_10_08.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\04_15_08.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\04_20_08.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\04_25_09.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\04_30_09.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\04_35_09.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\04_40_09.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\04_45_08.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\04_50_08.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\04_55_18.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\05_00_09.json
Processing ./data/raw/trunc/Fri

Processing ./data/raw/trunc/Fri_Apr_2_2021\17_50_08.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\17_55_10.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\18_00_09.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\18_05_08.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\18_12_18.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\18_15_10.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\18_20_07.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\18_25_10.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\18_30_10.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\18_35_09.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\18_40_09.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\18_45_08.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\18_50_07.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\18_55_09.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\19_00_09.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\19_05_19.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\19_10_08.json
Processing ./data/raw/trunc/Fri

Processing ./data/raw/trunc/Sat_Apr_3_2021\08_20_10.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\08_25_08.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\08_30_10.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\08_35_08.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\08_40_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\08_45_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\08_50_08.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\08_55_08.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\09_00_19.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\09_05_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\09_10_08.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\09_15_10.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\09_20_08.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\09_25_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\09_30_08.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\09_35_10.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\09_40_08.json
Processing ./data/raw/trunc/Sat

Processing ./data/raw/trunc/Sat_Apr_3_2021\23_30_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\23_35_18.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\23_40_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\23_45_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\23_50_10.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\23_55_09.json
Done


In [9]:
print(f"training input shape: {training_input.shape} training target shape: {training_target.shape}")
print(f"val input shape: {val_input.shape} val target shape: {val_target.shape}")
print(f"test input shape: {test_input.shape} test target shape: {test_target.shape}")
print(f"Adjacency matrix shape: {A.shape}")
print(f"Adjacency matrix shape: {A_wave.shape}")

training input shape: torch.Size([4491, 53, 6, 3]) training target shape: torch.Size([4491, 53, 2])
val input shape: torch.Size([1492, 53, 6, 3]) val target shape: torch.Size([1492, 53, 2])
test input shape: torch.Size([1493, 53, 6, 3]) test target shape: torch.Size([1493, 53, 2])
Adjacency matrix shape: (53, 53)
Adjacency matrix shape: torch.Size([53, 53])


In [5]:
print(metadata)
print(means)
print(stds)

{'B': 0, 'C': 1}
[3.993683   0.9056604  0.05673373]
[1.0052049  0.29230058 0.03545753]


In [7]:
# how to denormalize the speedband
denormalize(training_target, stds[0], means[0])

torch.Size([4491, 53, 2])